<a href="https://colab.research.google.com/github/ditumystro/Insurance_Data_Analysis/blob/main/final_investigate_tweetsCovid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers 
!pip install bertopic[all] 

In [ ]:
%%capture
import numpy as np
import csv
import urllib.request
import seaborn as sns
import pandas as pd
import re

from __future__ import print_function
import nltk
import os
import codecs
from sklearn import feature_extraction
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS
from scipy.cluster.hierarchy import ward, dendrogram
#from spellchecker import SpellChecker

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import string

from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from transformers import TokenClassificationPipeline
from datetime import date, timedelta 

from datetime import *
from dateutil import parser
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from collections import Counter
import random

random.seed(50)
keywords = ['mask','face covering', 'face cover', 'facecover']

In [ ]:
%%capture
def text_preprocessing(text):
  text = [re.sub('\S*@\S*\s?', '', sent) for sent in text]
  text = [re.sub('(@.*?)[\s]', '', sent) for sent in text]
  text = [re.sub('\s+', ' ', sent) for sent in text]
  text = [re.sub('http[s]?://\S+', '', sent) for sent in text]
# text = [re.sub(r'[0-9]+', '', sent) for sent in text]
  return text 

In [ ]:
%%capture
!wget https://www.dropbox.com/s/oal4ia7oae1d5dh/stage_two_df.csv
stage_two_df = pd.read_csv("stage_two_df.csv", sep=',')
!rm stage_two_df.csv


###Consider only retweeted tweets to shrink the amount of data

###Duplicate this script  (1) run stage data with only retweeted tweets (2) run bi-monthly datasets

In [ ]:
stage_two_df.shape

In [ ]:
stage_two_df = stage_two_df[stage_two_df['col37'] > 0] 
stage_two_df.shape 

In [ ]:
def extract_hashtags(text):
  regex = "#(\w+)"
  hashtag_list = re.findall(regex, text)
  if len(hashtag_list) > 0: hashtag_list = hashtag_list
  return hashtag_list 

In [ ]:
bio_description = stage_two_df.col21.to_list()
bio_description = text_preprocessing(bio_description)

tweets = stage_two_df.col4.to_list() 

bio_users = stage_two_df.col19.to_list()

In [ ]:
#@title Extracting hashtags from bio description
hashtag_df = pd.DataFrame()

for i in range(len(bio_description)):
  bio = bio_description[i]
  bio = extract_hashtags(bio)
  
  data = pd.DataFrame(bio)
  hashtag_df = hashtag_df.append(data)    

In [ ]:
hashtag_df.columns = ['hashtag']
hashtag_df['hashtag'] = hashtag_df['hashtag'].str.lower()

In [ ]:
LOC = hashtag_df.groupby(['hashtag']).size().to_frame('count').reset_index()
LOC = LOC.sort_values("count", ascending=False)

LOC.shape

In [ ]:
LOC_hd = LOC.head(200)

In [ ]:
sns.catplot(x="count", y="hashtag", kind="bar", height=28.27, aspect=11.7/28.27, data=LOC_hd)

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
nltk.download('punkt')

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    stemmer = SnowballStemmer("english")
    tokens = [word for sent in nltk.sent_tokenize(
        text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []

    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]

    return stems

def tokenize_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text)
              for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens


totalvocab_stemmed = []
totalvocab_tokenized = [] 

In [ ]:
for i in bio_description:
  replace_punctuation = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  i = i.translate(replace_punctuation)
  allwords_stemmed = tokenize_and_stem(i)
  totalvocab_stemmed.extend(allwords_stemmed)
  allwords_tokenized = tokenize_only(i)
  totalvocab_tokenized.extend(allwords_tokenized) 

In [ ]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index=totalvocab_stemmed)

In [ ]:
%%capture
tfidf_vectorizer = TfidfVectorizer(stop_words='english',tokenizer=tokenize_and_stem)
tfidf_matrix = tfidf_vectorizer.fit_transform(bio_description)

print(tfidf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names() 

In [ ]:
%%script echo skipping
#Vary number of clusters: 5, 10, 15, 20
nclusters = [5,10,15,20]
for clst in range(len(nclusters)):
  nclst = nclusters[clst]

  print("Num clusters: %d" % nclst, end='')
  print()
  print()

  km = KMeans(n_clusters=nclst)
  km.fit(tfidf_matrix)
  clusters = km.labels_.tolist()
  
  order_centroids = km.cluster_centers_.argsort()[:, ::-1]
  for i in range(nclst):
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :20]: 
      print(' %s' % terms[ind], end='')
      print()
    print()

In [ ]:
#@title Number clusters = 5
num_clusters = 5
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(num_clusters):
  print("Cluster %d words:" % i, end='')
  
  for ind in order_centroids[i, :20]: 
    print(' %s' % terms[ind], end='')
  print()

In [ ]:
#@title Number of users by cluster
df = {'bio': bio_description, 'cluster': clusters, 'user': bio_users, 'tweets': tweets}
frame = pd.DataFrame(df, columns=['bio','cluster', 'user', 'tweets'])

cluster_df = frame.groupby(['cluster']).size().to_frame('count').reset_index()
cluster_df = cluster_df.sort_values("count", ascending=False)
sns.catplot(x="cluster", y="count", kind="bar", data=cluster_df) 

In [ ]:
#@title Number clusters = 10
num_clusters = 10
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(num_clusters):
  print("Cluster %d words:" % i, end='')
  
  for ind in order_centroids[i, :20]: 
    print(' %s' % terms[ind], end='')
  print()

In [ ]:
#@title Number of users by cluster
df = {'bio': bio_description, 'cluster': clusters, 'user': bio_users, 'tweets': tweets}
frame = pd.DataFrame(df, columns=['bio','cluster', 'user', 'tweets'])

cluster_df = frame.groupby(['cluster']).size().to_frame('count').reset_index()
cluster_df = cluster_df.sort_values("count", ascending=False)
sns.catplot(x="cluster", y="count", kind="bar", data=cluster_df) 

In [ ]:
#@title Number clusters = 15
num_clusters = 15
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(num_clusters):
  print("Cluster %d words:" % i, end='')
  
  for ind in order_centroids[i, :20]: 
    print(' %s' % terms[ind], end='')
  print() 

In [ ]:
#@title Number of users by cluster
df = {'bio': bio_description, 'cluster': clusters, 'user': bio_users, 'tweets': tweets}
frame = pd.DataFrame(df, columns=['bio','cluster', 'user', 'tweets'])

cluster_df = frame.groupby(['cluster']).size().to_frame('count').reset_index()
cluster_df = cluster_df.sort_values("count", ascending=False)
sns.catplot(x="cluster", y="count", kind="bar", data=cluster_df) 

In [ ]:
#@title Number clusters = 20
num_clusters = 20
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(num_clusters):
  print("Cluster %d words:" % i, end='')
  
  for ind in order_centroids[i, :20]: 
    print(' %s' % terms[ind], end='')
  print() 

In [ ]:
#@title Number of users by cluster
df = {'bio': bio_description, 'cluster': clusters, 'user': bio_users, 'tweets': tweets}
frame = pd.DataFrame(df, columns=['bio','cluster', 'user', 'tweets'])

cluster_df = frame.groupby(['cluster']).size().to_frame('count').reset_index()
cluster_df = cluster_df.sort_values("count", ascending=False)
sns.catplot(x="cluster", y="count", kind="bar", data=cluster_df) 

In [ ]:
%%script echo skipping
from sklearn import metrics
import matplotlib.pyplot as plt

% matplotlib inline

x = []
y = []
yerr = []
nclusters = [5,10,15,20]

for i in range(len(nclusters)):
    n_clusters = nclusters[i]
    km = KMeans(n_clusters=n_clusters)
    
    sscore = []
    for i in range(1, 2):
        km.fit(tfidf_matrix)
        sscore.append(metrics.silhouette_score(tfidf_matrix, km.labels_))
    
    x.append(n_clusters)
    y.append(np.median(sscore))
    yerr.append(np.std(sscore))

plt.figure(figsize=(12, 6))
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score value')
plt.errorbar(x, y, yerr=yerr, fmt='-o', ecolor='g', color='g')
plt.xlim(xmin=5, xmax=20)
plt.ylim(ymin=0, ymax=0.015)
plt.show()

In [ ]:
#@title Applying BERTopic on Tweets stemming from users in Cluster 18
cluster_df_sub = frame[frame['cluster'].isin(['1'])]
clt_bio_users = cluster_df_sub.user.to_list()

tsv_read_clst = stage_two_df[stage_two_df['col19'].isin(clt_bio_users)]


In [ ]:
#@title Loading the NER pipeline by using a BERT based model and tokenizer.
bert_model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") 
ner_model = TokenClassificationPipeline( model = bert_model, tokenizer = tokenizer, grouped_entities=True)

In [ ]:
def manage_timestamps(text):
  text = parser.parse(text)
  return text.strftime('%Y-%m-%d %H:%M:%S')

def manage_dt(text):
  text = parser.parse(text)
  return text.strftime('%Y-%m-%d')

def manage_dates(text):
  return text.date() 

In [ ]:
%%capture
tsv_read_clst['timestamp'] = tsv_read_clst.col1.apply(manage_timestamps)
tsv_read_clst.col4 = tsv_read_clst.apply(lambda row: re.sub(r"http\S+", "", row.col4).lower(), 1)
tsv_read_clst.col4 = tsv_read_clst.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.col4.split())), 1)
tsv_read_clst.col4 = tsv_read_clst.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.col4).split()), 1)

timestamps = tsv_read_clst.timestamp.to_list()
tweets = tsv_read_clst.col4.to_list() 

In [ ]:
tsv_read_clst.shape 

In [ ]:
vec = CountVectorizer(stop_words='english', ngram_range=(1, 1))
model = BERTopic(verbose=True, n_gram_range=(1, 1), calculate_probabilities=True, min_topic_size=15, vectorizer_model=vec)
topics, _ = model.fit_transform(tweets) 

In [ ]:
topics_over_time = model.topics_over_time(tweets, topics, timestamps) 

In [ ]:
#@title Visualizing topics
model.visualize_topics() 

In [ ]:
topics_ot = topics_over_time
topics_ot['Date'] = topics_ot.Timestamp.apply(manage_dates)

topics_ot 

In [ ]:
over_time = topics_ot.groupby(['Topic', 'Date'])['Frequency'].sum().reset_index()
topics_ot = topics_ot.groupby(['Topic'])['Frequency'].sum().reset_index()
topics_ot.sort_values("Frequency", ascending = False).head(10) 

In [ ]:
#@title Top 10 topics.
topics_ot = topics_ot.sort_values("Frequency", ascending = False)

topics_ot['C'] = topics_ot.reset_index().index
topics_ftd = topics_ot.loc[~(topics_ot['Topic'] == -1)]


topics_filtered = topics_ftd[topics_ftd['C'].isin([0,1,2,3,4,5,6,7,8,9,10])]
topics_filtered = topics_filtered.Topic.to_list()
otf = over_time.Topic.isin(topics_filtered)
over_time_filtered = over_time[otf]

sns.set(rc={'figure.figsize':(18.7,8.27)})


sns.lineplot(x="Date",
             y="Frequency", 
             data = over_time_filtered, 
             hue="Topic", 
             palette="Dark2") 